In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def get_species_list(num_species=-1):
    #print("getting species list")
    #get page of all xento-canto species
    url = 'https://xeno-canto.org/collection/species/all'
    page = requests.get(url)
    soup = BeautifulSoup(page.text)
    species_table = soup.find_all("table", {"class": "results"})[0]
    
    #get headers of table
    headers = []
    for i in species_table.find_all('thead'):
         title = i.text
         headers.append(title)
    headers = headers[0].strip().split("\n")
    
    #add header for url and create DF
    headers.append('url')
    species_list = pd.DataFrame(columns = headers)
    
    #Get data from each row
    for j in species_table.find_all('tr'):
         #get row data
         row_data = j.find_all('td')
        
         #get all text from row
         row = [i.text for i in row_data]
        
         #get the link
         row.append(row_data[0].a["href"])
        
         #add to dataframe
         length = len(species_list)
         species_list.loc[length] = row

    #Decide to return full list or do RS     
    if (num_species == -1 or num_species > species_list.shape[0]):
        return species_list
    return species_list.sample(num_species).reset_index(drop=True)

#get_species_list()
#test1 = get_species_list(5)
#test1

In [3]:
def get_species_data(link_to_XC):
    #print("getting species file metadata for: " + link_to_XC)
    #get soup for Species
    url = 'https://xeno-canto.org' + link_to_XC
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    species_table = soup.find_all("table", {"class": "results"})[0]
    
    # Obtain every title of columns with tag <th>
    headers = []
    for i in species_table.find_all('thead'):
     title = i.text
     headers.append(title)

    headers = headers[0].strip()
    headers = headers.split("\n")
    headers[1] = "Common name / Scientific"
    headers.append("copyright")
    headers.append("filename")
    headers.append("download url")


    species_list = pd.DataFrame(columns = headers[1:])
    
    
    page_count = 1
    while len(species_table.find_all('tr')) != 1:
        
        #print(len(species_table.find_all('tr')))
        
        #Get data from each row
        for j in species_table.find_all('tr'):
            #get row data
            row_data = j.find_all('td')

            if (row_data == []):
                   continue

            #print(row_data) 
            #get all text from row
            row = [i.text.strip() for i in row_data][1:]

            #get copyright, download link and filename
            copyright = row_data[12].a.span["title"]
            try:
                 filename = row_data[11].a["download"]
            except:
                filename = "PROTECTED SPECIES"
            download_url = row_data[11].a["href"]
            row.append(copyright)
            row.append(filename)
            row.append(download_url)

            #add to dataframe
            length = len(species_list)
            species_list.loc[length] = row
        
        ## Get data for next page of audio data
        page_count += 1
        url = 'https://xeno-canto.org' + link_to_XC + "?pg=" + str(page_count)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'lxml')
        species_table = soup.find_all("table", {"class": "results"})[0]
    return species_list

#test2 = get_species_data(test1.iloc[0]["url"])
#test2

In [4]:
def download_by_url(file_data, download_folder):
    print(file_data)
    url = file_data[1]
    url = 'https://xeno-canto.org' + url
    r = requests.get(url, allow_redirects=True)
    path = download_folder+file_data[0]
    open(path, "wb").write(r.content)
    return path

def download_data(species_df, num_of_files, download_folder):
    print("downloading data")
    if (num_of_files != -1 and species_df.shape[0] > num_of_files):
        species_df = species_df.sample(num_of_files, replace=False)
    species_df["file_location"] = species_df[["filename","download url"]].apply(download_by_url,axis=1, args=(tuple([download_folder])))
    return species_df

#download_data(test2, 100, "./data/")

In [5]:
def download_xento_canto_data(num_species, num_of_files, download_folder, species_list=pd.DataFrame()):
    file_df = pd.DataFrame()
    if (species_list.empty):
        species_list = get_species_list(num_species)
    for url in species_list["url"]:
        try:
            species_files = get_species_data(url)
            temp_file_df = download_data(species_files, num_of_files, download_folder)
            if (file_df.empty):
                file_df = temp_file_df.reset_index(drop=True)
            else:
                file_df = file_df.append(temp_file_df.reset_index(drop=True)).reset_index(drop=True)
        except Exception as e:
            print(url + " did not download")
            print(e)
    return file_df
        

In [6]:
#metadata = download_xento_canto_data(-1, -1, "./data/")
#metadata

In [7]:
#metadata.to_csv("./metadata")

In [8]:
#get_species_list()
test1 = get_species_list(-1)
test1

,Common name,Scientific name,Status,No.,No. Back,url
0,\n\nCommon Ostrich\n\n,Struthio camelus,,7,0,/species/Struthio-camelus
1,\n\nSomali Ostrich\n\n,Struthio molybdophanes,,1,0,/species/Struthio-molybdophanes
2,\n\nGreater Rhea\n\n,Rhea americana,,23,1,/species/Rhea-americana
3,\n\nLesser Rhea\n\n,Rhea pennata,,4,0,/species/Rhea-pennata
4,\n\nSouthern Brown Kiwi\n\n,Apteryx australis,,3,0,/species/Apteryx-australis
...,...,...,...,...,...,...
10995,\n\nDanube Wide-winged Bush-cricket\n\n,Zeuneriana amplipennis,,1,0,/species/Zeuneriana-amplipennis
10996,\n\nIbera Seedeater\n\n,Sporophila digiacomoi,,44,0,/species/Sporophila-digiacomoi
10997,\n\nWhite-tailed Cisticola\n\n,Cisticola tax.nov.kilombero_2,,37,0,/species/Cisticola-tax.nov.kilombero_2
10998,\n\nKilombero Cisticola\n\n,Cisticola tax.nov.kilombero_1,,91,1,/species/Cisticola-tax.nov.kilombero_1


In [9]:
!pip install tqdm
from tqdm import tqdm
file_df = pd.DataFrame()
for url in tqdm(test1["url"]):
    try:
        species_files = get_species_data(url)
    except:
        continue
    if (file_df.empty):
        file_df = species_files.reset_index(drop=True)
    else:
        file_df = file_df.append(species_files.reset_index(drop=True)).reset_index(drop=True)
file_df

100%|█████████████████████████████████████████████████████████████████████████| 11000/11000 [11:10:06<00:00,  3.66s/it]


,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,Actions,Cat.nr.,copyright,filename,download url
0,Common Ostrich (Struthio camelus australis),0:53,Frank Lambert,2019-10-30,08:05,South Africa,"Polokwane Game Reserve, Polokwane, Limpopo",1300,"call, juvenile","A small juvenile, evidently separated from its...",ABCDE,XC516153,Creative Commons Attribution-NonCommercial-NoD...,XC516153 - Common Ostrich - Struthio camelus a...,/516153/download
1,Common Ostrich (Struthio camelus),0:26,Jeremy Hegge,2014-11-20,04:00,South Africa,"Mmabolela Reserve, Limpopo",750,call,Recording modified: Frequencies above 640hz re...,ABCDE,XC208209,Creative Commons Attribution-NonCommercial-Sha...,XC208209 - Common Ostrich - Struthio camelus.mp3,/208209/download
2,Common Ostrich (Struthio camelus),0:04,Jeremy Hegge,2014-11-21,06:00,South Africa,"Mmabolela Reserve, Limpopo",750,call,Recording modified: Frequencies above 640hz br...,ABCDE,XC208128,Creative Commons Attribution-NonCommercial-Sha...,XC208128 - Common Ostrich - Struthio camelus.mp3,/208128/download
3,Common Ostrich (Struthio camelus),0:11,Derek Solomon,2010-02-09,07:00,South Africa,Hoedspruit,523,Call,African Rock Python hissing in foreground\n [s...,ABCDE,XC46725,Creative Commons Attribution-NonCommercial-NoD...,XC46725 - Common Ostrich - Struthio camelus.mp3,/46725/download
4,Common Ostrich (Struthio camelus),1:47,Morioka Zoological Park ZOOMO,2021-09-06,17:00,Japan,"Morioka Zoological Park ZOOMO, Morioka City, I...",260,"adult, female, voice during egg laying, zoo co...","Although this is a zoo collection, I share the...",ABCDE,XC675445,Creative Commons Attribution-NonCommercial-Sha...,XC675445 - Common Ostrich - Struthio camelus.mp3,/675445/download
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686325,Kilombero Cisticola (Cisticola tax.nov.kilombe...,1:58,Louis A. Hansen,2019-06-29,15:12,Tanzania,-,240,song,Sailing on the river in a dugout canoe.\n [al...,ABCDE,XC511336,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511336
686326,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:34,Louis A. Hansen,2019-06-29,12:00,Tanzania,-,245,song,Sailing on the river in a dugout canoe. Duet s...,ABCDE,XC511326,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511326
686327,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:21,Louis A. Hansen,2019-06-29,09:28,Tanzania,-,245,song,Sailing on the river in a dugout canoe. Duet s...,ABCDE,XC511308,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511308
686328,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:06,Louis A. Hansen,2019-06-29,10:56,Tanzania,-,245,call,Volume increase\nplayback-used:no\n [sono],ABCDE,XC515823,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=515823


In [10]:
madreDeDios_species = pd.read_csv("C:/Users/Siloux/Downloads/MadreDeDiosBirdsXCList - MadreDeDiosBirdsXCList.csv")

In [19]:
file_df.to_csv("./all_meta_data.csv")

In [21]:
file_df["Common"] = file_df["Common name / Scientific"].apply(lambda x: x.split(" ")[0] + " "+ x.split(" ")[1])
file_df

,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,Actions,Cat.nr.,copyright,filename,download url,Common
0,Common Ostrich (Struthio camelus australis),0:53,Frank Lambert,2019-10-30,08:05,South Africa,"Polokwane Game Reserve, Polokwane, Limpopo",1300,"call, juvenile","A small juvenile, evidently separated from its...",ABCDE,XC516153,Creative Commons Attribution-NonCommercial-NoD...,XC516153 - Common Ostrich - Struthio camelus a...,/516153/download,Common Ostrich
1,Common Ostrich (Struthio camelus),0:26,Jeremy Hegge,2014-11-20,04:00,South Africa,"Mmabolela Reserve, Limpopo",750,call,Recording modified: Frequencies above 640hz re...,ABCDE,XC208209,Creative Commons Attribution-NonCommercial-Sha...,XC208209 - Common Ostrich - Struthio camelus.mp3,/208209/download,Common Ostrich
2,Common Ostrich (Struthio camelus),0:04,Jeremy Hegge,2014-11-21,06:00,South Africa,"Mmabolela Reserve, Limpopo",750,call,Recording modified: Frequencies above 640hz br...,ABCDE,XC208128,Creative Commons Attribution-NonCommercial-Sha...,XC208128 - Common Ostrich - Struthio camelus.mp3,/208128/download,Common Ostrich
3,Common Ostrich (Struthio camelus),0:11,Derek Solomon,2010-02-09,07:00,South Africa,Hoedspruit,523,Call,African Rock Python hissing in foreground\n [s...,ABCDE,XC46725,Creative Commons Attribution-NonCommercial-NoD...,XC46725 - Common Ostrich - Struthio camelus.mp3,/46725/download,Common Ostrich
4,Common Ostrich (Struthio camelus),1:47,Morioka Zoological Park ZOOMO,2021-09-06,17:00,Japan,"Morioka Zoological Park ZOOMO, Morioka City, I...",260,"adult, female, voice during egg laying, zoo co...","Although this is a zoo collection, I share the...",ABCDE,XC675445,Creative Commons Attribution-NonCommercial-Sha...,XC675445 - Common Ostrich - Struthio camelus.mp3,/675445/download,Common Ostrich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686325,Kilombero Cisticola (Cisticola tax.nov.kilombe...,1:58,Louis A. Hansen,2019-06-29,15:12,Tanzania,-,240,song,Sailing on the river in a dugout canoe.\n [al...,ABCDE,XC511336,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511336,Kilombero Cisticola
686326,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:34,Louis A. Hansen,2019-06-29,12:00,Tanzania,-,245,song,Sailing on the river in a dugout canoe. Duet s...,ABCDE,XC511326,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511326,Kilombero Cisticola
686327,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:21,Louis A. Hansen,2019-06-29,09:28,Tanzania,-,245,song,Sailing on the river in a dugout canoe. Duet s...,ABCDE,XC511308,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511308,Kilombero Cisticola
686328,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:06,Louis A. Hansen,2019-06-29,10:56,Tanzania,-,245,call,Volume increase\nplayback-used:no\n [sono],ABCDE,XC515823,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=515823,Kilombero Cisticola


In [25]:
file_df.groupby("Common").count().sort_values(by="Length", ascending=False)

,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,Actions,Cat.nr.,copyright,filename,download url
Common,,,,,,,,,,,,,,,
Great Tit,6831,6831,6831,6831,6831,6831,6831,6831,6831,6831,6831,6831,6831,6831,6831
Common Blackbird,5909,5909,5909,5909,5909,5909,5909,5909,5909,5909,5909,5909,5909,5909,5909
Common Chaffinch,5546,5546,5546,5546,5546,5546,5546,5546,5546,5546,5546,5546,5546,5546,5546
Common Chiffchaff,5026,5026,5026,5026,5026,5026,5026,5026,5026,5026,5026,5026,5026,5026,5026
European Robin,4686,4686,4686,4686,4686,4686,4686,4686,4686,4686,4686,4686,4686,4686,4686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
(?) Red-backed,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Rennell White-eye,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Mauritius Kestrel,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [22]:
file_MDD_XC = file_df.merge(madreDeDios_species, left_on="Common", right_on="Common")
file_MDD_XC

,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,...,Order,Family,Genus,Epithet,Species,CommonES,Status,XCRecordings,VoucherMUSM-Orn,Priority
0,Grey Tinamou (Tinamus tao tao),0:50,GABRIEL LEITE,2020-10-13,06:30,Brazil,"Parauapebas, Pará",500,"adult, life stage uncertain, sex uncertain, song",bird-seen:no\nplayback-used:no\n [sono],...,Tinamiformes,Tinamidae,Tinamus,tao,Tinamus tao,Tinamú Tao,Vulnerable,54,amazoncam,1
1,Grey Tinamou (Tinamus tao),0:27,Fabrice Schmitt,2016-02-19,07:30,Colombia,"RNA El Dorado, Sierra de Santa Marta, Magdalena",2000,song,bird-seen:no\nplayback-used:no\n [sono],...,Tinamiformes,Tinamidae,Tinamus,tao,Tinamus tao,Tinamú Tao,Vulnerable,54,amazoncam,1
2,Grey Tinamou (Tinamus tao),3:43,Niels Krabbe,2007-02-05,07:20,Colombia,"Magdalena: San Lorenzo ridge, Sierra Nevada de...",1750,tremolo song,Humid forest. Krabbe (2008) cut 1. Tape refere...,...,Tinamiformes,Tinamidae,Tinamus,tao,Tinamus tao,Tinamú Tao,Vulnerable,54,amazoncam,1
3,Grey Tinamou (Tinamus tao),0:47,Jeremy Minns,2005-10-26,06:48,Brazil,Serra dos Carajás. Águas Claras,550,tremolo song,terra firme forest - same bird as recorded in ...,...,Tinamiformes,Tinamidae,Tinamus,tao,Tinamus tao,Tinamú Tao,Vulnerable,54,amazoncam,1
4,Grey Tinamou (Tinamus tao),0:04,A. Bennett Hennessey,1998-10-22,6:12,Bolivia,"Serranía Cuchilla, Pilón Lajas Biosphere Reser...",500 - 900,A single song,At about 900 m.\n [CD-ROM] [also]\n[sono],...,Tinamiformes,Tinamidae,Tinamus,tao,Tinamus tao,Tinamú Tao,Vulnerable,54,amazoncam,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93561,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:52,Eduardo Patrial,2013-10-15,10:00,Brazil,"Cristalino Jungle Lodge, MT",260,song,bird-seen:yes\nplayback-used:yes\n [sono],...,Passeriformes,Thraupidae,Parkerthraustes,humeralis,Parkerthraustes humeralis,Picogrueso Humeral,NaN,26,NaN,NaN
93562,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:09,A. Bennett Hennessey,1998-09-25,8:37,Bolivia,"Serranías Beu & Chepete, Pilón Lajas Biospher...",750 - 1450,call,Two birds. At 700 - 1000 m.\n [CD-ROM] [sono],...,Passeriformes,Thraupidae,Parkerthraustes,humeralis,Parkerthraustes humeralis,Picogrueso Humeral,NaN,26,NaN,NaN
93563,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:37,Roger Ahlman,2011-08-14,16:50,Ecuador,"Miazi, Zamora-Chinchipe",940,"call, song",After playback. Same bird as XC86226.\n [also...,...,Passeriformes,Thraupidae,Parkerthraustes,humeralis,Parkerthraustes humeralis,Picogrueso Humeral,NaN,26,NaN,NaN
93564,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:31,Huw Lloyd,2008-06-15,16:30,Peru,"Manu Wildlife Centre, Madre de Dios",320,Response to playback,Calls from three birds (possibly parents plus ...,...,Passeriformes,Thraupidae,Parkerthraustes,humeralis,Parkerthraustes humeralis,Picogrueso Humeral,NaN,26,NaN,NaN


In [23]:
file_MDD_XC.groupby("Common").count().sort_values(by="Length", ascending=False)

,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,...,Order,Family,Genus,Epithet,Species,CommonES,Status,XCRecordings,VoucherMUSM-Orn,Priority
Common,,,,,,,,,,,,,,,,,,,,,
House Sparrow,2262,2262,2262,2262,2262,2262,2262,2262,2262,2262,...,2262,2262,2262,2262,2262,2262,2262,2262,0,0
House Wren,1660,1660,1660,1660,1660,1660,1660,1660,1660,1660,...,1660,1660,1660,1660,1660,1660,0,1660,1660,0
Barn Swallow,1531,1531,1531,1531,1531,1531,1531,1531,1531,1531,...,1531,1531,1531,1531,1531,1531,0,1531,0,0
Rufous-collared Sparrow,794,794,794,794,794,794,794,794,794,794,...,794,794,794,794,794,794,0,794,0,0
Rufous-browed Peppershrike,745,745,745,745,745,745,745,745,745,745,...,745,745,745,745,745,745,0,745,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
King Vulture,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,0,2,0,0
Koepcke's Hermit,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,0
Manu Parrotlet,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,0,0


In [24]:
test1.set_index("Common name").loc["\n\nOrange-cheeked Parrot\n\n"]

Scientific name             Pyrilia barrabandi
Status                                        
No.                                         33
No. Back                                    19
url                /species/Pyrilia-barrabandi
Name: \n\nOrange-cheeked Parrot\n\n, dtype: object

In [ ]:
#url = "https://xeno-canto.org/species/Pyrilia-barrabandi"
#get_species_data("/species/Pyrilia-barrabandi")

In [26]:
file_df2 = pd.read_csv("./all_meta_data.csv")
file_df2

,Unnamed: 0,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,Actions,Cat.nr.,copyright,filename,download url,Common
0,0,Common Ostrich (Struthio camelus australis),0:53,Frank Lambert,2019-10-30,08:05,South Africa,"Polokwane Game Reserve, Polokwane, Limpopo",1300,"call, juvenile","A small juvenile, evidently separated from its...",ABCDE,XC516153,Creative Commons Attribution-NonCommercial-NoD...,XC516153 - Common Ostrich - Struthio camelus a...,/516153/download,Common Ostrich
1,1,Common Ostrich (Struthio camelus),0:26,Jeremy Hegge,2014-11-20,04:00,South Africa,"Mmabolela Reserve, Limpopo",750,call,Recording modified: Frequencies above 640hz re...,ABCDE,XC208209,Creative Commons Attribution-NonCommercial-Sha...,XC208209 - Common Ostrich - Struthio camelus.mp3,/208209/download,Common Ostrich
2,2,Common Ostrich (Struthio camelus),0:04,Jeremy Hegge,2014-11-21,06:00,South Africa,"Mmabolela Reserve, Limpopo",750,call,Recording modified: Frequencies above 640hz br...,ABCDE,XC208128,Creative Commons Attribution-NonCommercial-Sha...,XC208128 - Common Ostrich - Struthio camelus.mp3,/208128/download,Common Ostrich
3,3,Common Ostrich (Struthio camelus),0:11,Derek Solomon,2010-02-09,07:00,South Africa,Hoedspruit,523,Call,African Rock Python hissing in foreground\n [s...,ABCDE,XC46725,Creative Commons Attribution-NonCommercial-NoD...,XC46725 - Common Ostrich - Struthio camelus.mp3,/46725/download,Common Ostrich
4,4,Common Ostrich (Struthio camelus),1:47,Morioka Zoological Park ZOOMO,2021-09-06,17:00,Japan,"Morioka Zoological Park ZOOMO, Morioka City, I...",260,"adult, female, voice during egg laying, zoo co...","Although this is a zoo collection, I share the...",ABCDE,XC675445,Creative Commons Attribution-NonCommercial-Sha...,XC675445 - Common Ostrich - Struthio camelus.mp3,/675445/download,Common Ostrich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686325,686325,Kilombero Cisticola (Cisticola tax.nov.kilombe...,1:58,Louis A. Hansen,2019-06-29,15:12,Tanzania,-,240,song,Sailing on the river in a dugout canoe.\n [al...,ABCDE,XC511336,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511336,Kilombero Cisticola
686326,686326,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:34,Louis A. Hansen,2019-06-29,12:00,Tanzania,-,245,song,Sailing on the river in a dugout canoe. Duet s...,ABCDE,XC511326,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511326,Kilombero Cisticola
686327,686327,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:21,Louis A. Hansen,2019-06-29,09:28,Tanzania,-,245,song,Sailing on the river in a dugout canoe. Duet s...,ABCDE,XC511308,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511308,Kilombero Cisticola
686328,686328,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:06,Louis A. Hansen,2019-06-29,10:56,Tanzania,-,245,call,Volume increase\nplayback-used:no\n [sono],ABCDE,XC515823,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=515823,Kilombero Cisticola


In [27]:
file_df

,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,Actions,Cat.nr.,copyright,filename,download url,Common
0,Common Ostrich (Struthio camelus australis),0:53,Frank Lambert,2019-10-30,08:05,South Africa,"Polokwane Game Reserve, Polokwane, Limpopo",1300,"call, juvenile","A small juvenile, evidently separated from its...",ABCDE,XC516153,Creative Commons Attribution-NonCommercial-NoD...,XC516153 - Common Ostrich - Struthio camelus a...,/516153/download,Common Ostrich
1,Common Ostrich (Struthio camelus),0:26,Jeremy Hegge,2014-11-20,04:00,South Africa,"Mmabolela Reserve, Limpopo",750,call,Recording modified: Frequencies above 640hz re...,ABCDE,XC208209,Creative Commons Attribution-NonCommercial-Sha...,XC208209 - Common Ostrich - Struthio camelus.mp3,/208209/download,Common Ostrich
2,Common Ostrich (Struthio camelus),0:04,Jeremy Hegge,2014-11-21,06:00,South Africa,"Mmabolela Reserve, Limpopo",750,call,Recording modified: Frequencies above 640hz br...,ABCDE,XC208128,Creative Commons Attribution-NonCommercial-Sha...,XC208128 - Common Ostrich - Struthio camelus.mp3,/208128/download,Common Ostrich
3,Common Ostrich (Struthio camelus),0:11,Derek Solomon,2010-02-09,07:00,South Africa,Hoedspruit,523,Call,African Rock Python hissing in foreground\n [s...,ABCDE,XC46725,Creative Commons Attribution-NonCommercial-NoD...,XC46725 - Common Ostrich - Struthio camelus.mp3,/46725/download,Common Ostrich
4,Common Ostrich (Struthio camelus),1:47,Morioka Zoological Park ZOOMO,2021-09-06,17:00,Japan,"Morioka Zoological Park ZOOMO, Morioka City, I...",260,"adult, female, voice during egg laying, zoo co...","Although this is a zoo collection, I share the...",ABCDE,XC675445,Creative Commons Attribution-NonCommercial-Sha...,XC675445 - Common Ostrich - Struthio camelus.mp3,/675445/download,Common Ostrich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686325,Kilombero Cisticola (Cisticola tax.nov.kilombe...,1:58,Louis A. Hansen,2019-06-29,15:12,Tanzania,-,240,song,Sailing on the river in a dugout canoe.\n [al...,ABCDE,XC511336,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511336,Kilombero Cisticola
686326,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:34,Louis A. Hansen,2019-06-29,12:00,Tanzania,-,245,song,Sailing on the river in a dugout canoe. Duet s...,ABCDE,XC511326,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511326,Kilombero Cisticola
686327,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:21,Louis A. Hansen,2019-06-29,09:28,Tanzania,-,245,song,Sailing on the river in a dugout canoe. Duet s...,ABCDE,XC511308,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511308,Kilombero Cisticola
686328,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:06,Louis A. Hansen,2019-06-29,10:56,Tanzania,-,245,call,Volume increase\nplayback-used:no\n [sono],ABCDE,XC515823,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=515823,Kilombero Cisticola


In [31]:
import re
mixed_bird = pd.read_csv("C:/Users/Siloux/Desktop/E4E/PyHa/mixed_bird/mixed_bird_manual.csv")
mixed_bird["ID"] = mixed_bird["IN FILE"].apply(lambda x: re.findall(r'\d+', x)[0])
mixed_bird

,Unnamed: 0,IN FILE,CLIP LENGTH,OFFSET,DURATION,MAX FREQ,MIN FREQ,SAMPLE RATE,MANUAL ID,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FOLDER,ID
0,0,Aburria-aburri-257423.wav,24.6335,2.3614,3.3205,24000,0,44100,Aburria aburri Wattled Guan,1050.630,alarce,11/22/2021,2:13:53,./TEST/,257423
1,1,Aburria-aburri-257423.wav,24.6335,6.7320,2.0103,24000,0,44100,Aburria aburri Wattled Guan,1050.630,alarce,11/22/2021,2:13:53,./TEST/,257423
2,2,Aburria-aburri-257423.wav,24.6335,19.2438,3.4505,24000,0,44100,Aburria aburri Wattled Guan,1050.630,alarce,11/22/2021,2:13:53,./TEST/,257423
3,3,Aburria-aburri-387212.wav,84.4800,0.6011,3.0800,24000,0,44100,Aburria aburri Wattled Guan,184.388,alarce,11/22/2021,3:28:01,./TEST/,387212
4,4,Aburria-aburri-387212.wav,84.4800,77.6811,3.1400,24000,0,44100,Aburria aburri Wattled Guan,184.385,alarce,11/22/2021,3:28:01,./TEST/,387212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31360,31360,Zonotrichia-capensis-232609.wav,98.8560,63.2772,1.8301,24000,0,32000,Zonotrichia capensis Rufous-collared Sparrow,623.174,JacobGlennAyers,10/10/2021,2:06:40,./TEST/,232609
31361,31361,Zonotrichia-capensis-232609.wav,98.8560,34.1554,2.4702,24000,0,32000,Zonotrichia capensis Rufous-collared Sparrow,623.135,JacobGlennAyers,10/10/2021,2:06:40,./TEST/,232609
31362,31362,Zonotrichia-capensis-377483.wav,18.2596,0.5762,3.2199,24000,0,44100,Zonotrichia capensis Rufous-collared Sparrow,229.527,JacobGlennAyers,10/18/2021,17:33:36,./TEST/,377483
31363,31363,Zonotrichia-capensis-377483.wav,18.2596,6.8160,2.1500,24000,0,44100,Zonotrichia capensis Rufous-collared Sparrow,229.543,JacobGlennAyers,10/18/2021,17:33:36,./TEST/,377483


In [33]:
file_df["ID"] = file_df["Cat.nr."].apply(lambda x: re.findall(r'\d+', x)[0])

In [34]:
file_df

,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,Actions,Cat.nr.,copyright,filename,download url,Common,ID
0,Common Ostrich (Struthio camelus australis),0:53,Frank Lambert,2019-10-30,08:05,South Africa,"Polokwane Game Reserve, Polokwane, Limpopo",1300,"call, juvenile","A small juvenile, evidently separated from its...",ABCDE,XC516153,Creative Commons Attribution-NonCommercial-NoD...,XC516153 - Common Ostrich - Struthio camelus a...,/516153/download,Common Ostrich,516153
1,Common Ostrich (Struthio camelus),0:26,Jeremy Hegge,2014-11-20,04:00,South Africa,"Mmabolela Reserve, Limpopo",750,call,Recording modified: Frequencies above 640hz re...,ABCDE,XC208209,Creative Commons Attribution-NonCommercial-Sha...,XC208209 - Common Ostrich - Struthio camelus.mp3,/208209/download,Common Ostrich,208209
2,Common Ostrich (Struthio camelus),0:04,Jeremy Hegge,2014-11-21,06:00,South Africa,"Mmabolela Reserve, Limpopo",750,call,Recording modified: Frequencies above 640hz br...,ABCDE,XC208128,Creative Commons Attribution-NonCommercial-Sha...,XC208128 - Common Ostrich - Struthio camelus.mp3,/208128/download,Common Ostrich,208128
3,Common Ostrich (Struthio camelus),0:11,Derek Solomon,2010-02-09,07:00,South Africa,Hoedspruit,523,Call,African Rock Python hissing in foreground\n [s...,ABCDE,XC46725,Creative Commons Attribution-NonCommercial-NoD...,XC46725 - Common Ostrich - Struthio camelus.mp3,/46725/download,Common Ostrich,46725
4,Common Ostrich (Struthio camelus),1:47,Morioka Zoological Park ZOOMO,2021-09-06,17:00,Japan,"Morioka Zoological Park ZOOMO, Morioka City, I...",260,"adult, female, voice during egg laying, zoo co...","Although this is a zoo collection, I share the...",ABCDE,XC675445,Creative Commons Attribution-NonCommercial-Sha...,XC675445 - Common Ostrich - Struthio camelus.mp3,/675445/download,Common Ostrich,675445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686325,Kilombero Cisticola (Cisticola tax.nov.kilombe...,1:58,Louis A. Hansen,2019-06-29,15:12,Tanzania,-,240,song,Sailing on the river in a dugout canoe.\n [al...,ABCDE,XC511336,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511336,Kilombero Cisticola,511336
686326,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:34,Louis A. Hansen,2019-06-29,12:00,Tanzania,-,245,song,Sailing on the river in a dugout canoe. Duet s...,ABCDE,XC511326,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511326,Kilombero Cisticola,511326
686327,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:21,Louis A. Hansen,2019-06-29,09:28,Tanzania,-,245,song,Sailing on the river in a dugout canoe. Duet s...,ABCDE,XC511308,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511308,Kilombero Cisticola,511308
686328,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:06,Louis A. Hansen,2019-06-29,10:56,Tanzania,-,245,call,Volume increase\nplayback-used:no\n [sono],ABCDE,XC515823,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=515823,Kilombero Cisticola,515823


In [40]:
file_df[~file_df["ID"].isin(mixed_bird["ID"])]

,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,Actions,Cat.nr.,copyright,filename,download url,Common,ID
0,Common Ostrich (Struthio camelus australis),0:53,Frank Lambert,2019-10-30,08:05,South Africa,"Polokwane Game Reserve, Polokwane, Limpopo",1300,"call, juvenile","A small juvenile, evidently separated from its...",ABCDE,XC516153,Creative Commons Attribution-NonCommercial-NoD...,XC516153 - Common Ostrich - Struthio camelus a...,/516153/download,Common Ostrich,516153
1,Common Ostrich (Struthio camelus),0:26,Jeremy Hegge,2014-11-20,04:00,South Africa,"Mmabolela Reserve, Limpopo",750,call,Recording modified: Frequencies above 640hz re...,ABCDE,XC208209,Creative Commons Attribution-NonCommercial-Sha...,XC208209 - Common Ostrich - Struthio camelus.mp3,/208209/download,Common Ostrich,208209
2,Common Ostrich (Struthio camelus),0:04,Jeremy Hegge,2014-11-21,06:00,South Africa,"Mmabolela Reserve, Limpopo",750,call,Recording modified: Frequencies above 640hz br...,ABCDE,XC208128,Creative Commons Attribution-NonCommercial-Sha...,XC208128 - Common Ostrich - Struthio camelus.mp3,/208128/download,Common Ostrich,208128
3,Common Ostrich (Struthio camelus),0:11,Derek Solomon,2010-02-09,07:00,South Africa,Hoedspruit,523,Call,African Rock Python hissing in foreground\n [s...,ABCDE,XC46725,Creative Commons Attribution-NonCommercial-NoD...,XC46725 - Common Ostrich - Struthio camelus.mp3,/46725/download,Common Ostrich,46725
4,Common Ostrich (Struthio camelus),1:47,Morioka Zoological Park ZOOMO,2021-09-06,17:00,Japan,"Morioka Zoological Park ZOOMO, Morioka City, I...",260,"adult, female, voice during egg laying, zoo co...","Although this is a zoo collection, I share the...",ABCDE,XC675445,Creative Commons Attribution-NonCommercial-Sha...,XC675445 - Common Ostrich - Struthio camelus.mp3,/675445/download,Common Ostrich,675445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686325,Kilombero Cisticola (Cisticola tax.nov.kilombe...,1:58,Louis A. Hansen,2019-06-29,15:12,Tanzania,-,240,song,Sailing on the river in a dugout canoe.\n [al...,ABCDE,XC511336,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511336,Kilombero Cisticola,511336
686326,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:34,Louis A. Hansen,2019-06-29,12:00,Tanzania,-,245,song,Sailing on the river in a dugout canoe. Duet s...,ABCDE,XC511326,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511326,Kilombero Cisticola,511326
686327,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:21,Louis A. Hansen,2019-06-29,09:28,Tanzania,-,245,song,Sailing on the river in a dugout canoe. Duet s...,ABCDE,XC511308,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=511308,Kilombero Cisticola,511308
686328,Kilombero Cisticola (Cisticola tax.nov.kilombe...,0:06,Louis A. Hansen,2019-06-29,10:56,Tanzania,-,245,call,Volume increase\nplayback-used:no\n [sono],ABCDE,XC515823,Creative Commons Attribution-NonCommercial-Sha...,PROTECTED SPECIES,/new_thread.php?frontID=515823,Kilombero Cisticola,515823


In [41]:
file_df[file_df["Common"].isin(madreDeDios_species["Common"])]

,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,Actions,Cat.nr.,copyright,filename,download url,Common,ID
120,Grey Tinamou (Tinamus tao tao),0:50,GABRIEL LEITE,2020-10-13,06:30,Brazil,"Parauapebas, Pará",500,"adult, life stage uncertain, sex uncertain, song",bird-seen:no\nplayback-used:no\n [sono],ABCDE,XC597722,Creative Commons Attribution-NonCommercial-Sha...,XC597722 - Grey Tinamou - Tinamus tao tao.mp3,/597722/download,Grey Tinamou,597722
121,Grey Tinamou (Tinamus tao),0:27,Fabrice Schmitt,2016-02-19,07:30,Colombia,"RNA El Dorado, Sierra de Santa Marta, Magdalena",2000,song,bird-seen:no\nplayback-used:no\n [sono],ABCDE,XC318739,Creative Commons Attribution-NonCommercial-Sha...,XC318739 - Grey Tinamou - Tinamus tao.mp3,/318739/download,Grey Tinamou,318739
122,Grey Tinamou (Tinamus tao),3:43,Niels Krabbe,2007-02-05,07:20,Colombia,"Magdalena: San Lorenzo ridge, Sierra Nevada de...",1750,tremolo song,Humid forest. Krabbe (2008) cut 1. Tape refere...,ABCDE,XC236216,Creative Commons Attribution-NonCommercial-Sha...,XC236216 - Grey Tinamou - Tinamus tao.mp3,/236216/download,Grey Tinamou,236216
123,Grey Tinamou (Tinamus tao),0:47,Jeremy Minns,2005-10-26,06:48,Brazil,Serra dos Carajás. Águas Claras,550,tremolo song,terra firme forest - same bird as recorded in ...,ABCDE,XC217695,Creative Commons Attribution-NonCommercial-Sha...,XC217695 - Grey Tinamou - Tinamus tao.mp3,/217695/download,Grey Tinamou,217695
124,Grey Tinamou (Tinamus tao),0:04,A. Bennett Hennessey,1998-10-22,6:12,Bolivia,"Serranía Cuchilla, Pilón Lajas Biosphere Reser...",500 - 900,A single song,At about 900 m.\n [CD-ROM] [also]\n[sono],ABCDE,XC3145,Creative Commons Attribution-NonCommercial-NoD...,XC3145 - Grey Tinamou - Tinamus tao.mp3,/3145/download,Grey Tinamou,3145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686010,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:52,Eduardo Patrial,2013-10-15,10:00,Brazil,"Cristalino Jungle Lodge, MT",260,song,bird-seen:yes\nplayback-used:yes\n [sono],ABCDE,XC255003,Creative Commons Attribution-NonCommercial-Sha...,XC255003 - Yellow-shouldered Grosbeak - Parker...,/255003/download,Yellow-shouldered Grosbeak,255003
686011,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:09,A. Bennett Hennessey,1998-09-25,8:37,Bolivia,"Serranías Beu & Chepete, Pilón Lajas Biospher...",750 - 1450,call,Two birds. At 700 - 1000 m.\n [CD-ROM] [sono],ABCDE,XC3123,Creative Commons Attribution-NonCommercial-NoD...,XC3123 - Yellow-shouldered Grosbeak - Parkerth...,/3123/download,Yellow-shouldered Grosbeak,3123
686012,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:37,Roger Ahlman,2011-08-14,16:50,Ecuador,"Miazi, Zamora-Chinchipe",940,"call, song",After playback. Same bird as XC86226.\n [also...,ABCDE,XC93438,Creative Commons Attribution-NonCommercial-NoD...,XC93438 - Yellow-shouldered Grosbeak - Parkert...,/93438/download,Yellow-shouldered Grosbeak,93438
686013,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:31,Huw Lloyd,2008-06-15,16:30,Peru,"Manu Wildlife Centre, Madre de Dios",320,Response to playback,Calls from three birds (possibly parents plus ...,ABCDE,XC36455,Creative Commons Attribution-NonCommercial-NoD...,XC36455 - Yellow-shouldered Grosbeak - Parkert...,/36455/download,Yellow-shouldered Grosbeak,36455


In [51]:
mixed_bird["MANUAL ID LENGTH"] = mixed_bird["MANUAL ID"].apply(lambda x: len(x.split(" ")))
mixed_bird["Common"] = mixed_bird["MANUAL ID"].apply(lambda x: " ".join(x.split(" ")[2:]))
mixed_bird

,Unnamed: 0,IN FILE,CLIP LENGTH,OFFSET,DURATION,MAX FREQ,MIN FREQ,SAMPLE RATE,MANUAL ID,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FOLDER,ID,MANUAL ID LENGTH,Common
0,0,Aburria-aburri-257423.wav,24.6335,2.3614,3.3205,24000,0,44100,Aburria aburri Wattled Guan,1050.630,alarce,11/22/2021,2:13:53,./TEST/,257423,4,Wattled Guan
1,1,Aburria-aburri-257423.wav,24.6335,6.7320,2.0103,24000,0,44100,Aburria aburri Wattled Guan,1050.630,alarce,11/22/2021,2:13:53,./TEST/,257423,4,Wattled Guan
2,2,Aburria-aburri-257423.wav,24.6335,19.2438,3.4505,24000,0,44100,Aburria aburri Wattled Guan,1050.630,alarce,11/22/2021,2:13:53,./TEST/,257423,4,Wattled Guan
3,3,Aburria-aburri-387212.wav,84.4800,0.6011,3.0800,24000,0,44100,Aburria aburri Wattled Guan,184.388,alarce,11/22/2021,3:28:01,./TEST/,387212,4,Wattled Guan
4,4,Aburria-aburri-387212.wav,84.4800,77.6811,3.1400,24000,0,44100,Aburria aburri Wattled Guan,184.385,alarce,11/22/2021,3:28:01,./TEST/,387212,4,Wattled Guan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31360,31360,Zonotrichia-capensis-232609.wav,98.8560,63.2772,1.8301,24000,0,32000,Zonotrichia capensis Rufous-collared Sparrow,623.174,JacobGlennAyers,10/10/2021,2:06:40,./TEST/,232609,4,Rufous-collared Sparrow
31361,31361,Zonotrichia-capensis-232609.wav,98.8560,34.1554,2.4702,24000,0,32000,Zonotrichia capensis Rufous-collared Sparrow,623.135,JacobGlennAyers,10/10/2021,2:06:40,./TEST/,232609,4,Rufous-collared Sparrow
31362,31362,Zonotrichia-capensis-377483.wav,18.2596,0.5762,3.2199,24000,0,44100,Zonotrichia capensis Rufous-collared Sparrow,229.527,JacobGlennAyers,10/18/2021,17:33:36,./TEST/,377483,4,Rufous-collared Sparrow
31363,31363,Zonotrichia-capensis-377483.wav,18.2596,6.8160,2.1500,24000,0,44100,Zonotrichia capensis Rufous-collared Sparrow,229.543,JacobGlennAyers,10/18/2021,17:33:36,./TEST/,377483,4,Rufous-collared Sparrow


In [42]:
file_df[(file_df["Common"].isin(madreDeDios_species["Common"])) & (~file_df["ID"].isin(mixed_bird["ID"]))]

,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,Actions,Cat.nr.,copyright,filename,download url,Common,ID
120,Grey Tinamou (Tinamus tao tao),0:50,GABRIEL LEITE,2020-10-13,06:30,Brazil,"Parauapebas, Pará",500,"adult, life stage uncertain, sex uncertain, song",bird-seen:no\nplayback-used:no\n [sono],ABCDE,XC597722,Creative Commons Attribution-NonCommercial-Sha...,XC597722 - Grey Tinamou - Tinamus tao tao.mp3,/597722/download,Grey Tinamou,597722
121,Grey Tinamou (Tinamus tao),0:27,Fabrice Schmitt,2016-02-19,07:30,Colombia,"RNA El Dorado, Sierra de Santa Marta, Magdalena",2000,song,bird-seen:no\nplayback-used:no\n [sono],ABCDE,XC318739,Creative Commons Attribution-NonCommercial-Sha...,XC318739 - Grey Tinamou - Tinamus tao.mp3,/318739/download,Grey Tinamou,318739
125,Grey Tinamou (Tinamus tao kleei),0:08,Richard E. Webster,2021-11-24,10:05,Bolivia,"Refugio Volcanes, Samaipata, Florida, Santa Cr...",1100,"adult, sex uncertain, song",bird-seen:no\nplayback-used:no\n [sono],ABCDE,XC708505,Creative Commons Attribution-NonCommercial-Sha...,XC708505 - Grey Tinamou - Tinamus tao kleei.mp3,/708505/download,Grey Tinamou,708505
126,Grey Tinamou (Tinamus tao),0:06,Dante Buzzetti,2008-08-04,19:53,Brazil,"Parque Estadual Chandless,Santa Rosa do Purus,...",265,song,"DCB 626-3e, two individuals singing\nbird-seen...",ABCDE,XC564679,Creative Commons Attribution-NonCommercial-Sha...,XC564679 - Grey Tinamou - Tinamus tao.mp3,/564679/download,Grey Tinamou,564679
127,Grey Tinamou (Tinamus tao),0:48,Dante Buzzetti,2008-08-04,18:53,Brazil,"Parque Estadual Chandless,Santa Rosa do Purus,...",265,song,"DCB 625-18a, two or three individuals singing....",ABCDE,XC557622,Creative Commons Attribution-NonCommercial-Sha...,XC557622 - Grey Tinamou - Tinamus tao.mp3,/557622/download,Grey Tinamou,557622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686010,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:52,Eduardo Patrial,2013-10-15,10:00,Brazil,"Cristalino Jungle Lodge, MT",260,song,bird-seen:yes\nplayback-used:yes\n [sono],ABCDE,XC255003,Creative Commons Attribution-NonCommercial-Sha...,XC255003 - Yellow-shouldered Grosbeak - Parker...,/255003/download,Yellow-shouldered Grosbeak,255003
686011,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:09,A. Bennett Hennessey,1998-09-25,8:37,Bolivia,"Serranías Beu & Chepete, Pilón Lajas Biospher...",750 - 1450,call,Two birds. At 700 - 1000 m.\n [CD-ROM] [sono],ABCDE,XC3123,Creative Commons Attribution-NonCommercial-NoD...,XC3123 - Yellow-shouldered Grosbeak - Parkerth...,/3123/download,Yellow-shouldered Grosbeak,3123
686012,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:37,Roger Ahlman,2011-08-14,16:50,Ecuador,"Miazi, Zamora-Chinchipe",940,"call, song",After playback. Same bird as XC86226.\n [also...,ABCDE,XC93438,Creative Commons Attribution-NonCommercial-NoD...,XC93438 - Yellow-shouldered Grosbeak - Parkert...,/93438/download,Yellow-shouldered Grosbeak,93438
686013,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:31,Huw Lloyd,2008-06-15,16:30,Peru,"Manu Wildlife Centre, Madre de Dios",320,Response to playback,Calls from three birds (possibly parents plus ...,ABCDE,XC36455,Creative Commons Attribution-NonCommercial-NoD...,XC36455 - Yellow-shouldered Grosbeak - Parkert...,/36455/download,Yellow-shouldered Grosbeak,36455


In [54]:
unlabeled_mixed_bird = file_df[(file_df["Common"].isin(madreDeDios_species["Common"])) & (~file_df["ID"].isin(mixed_bird["ID"])) & (file_df["Common"].isin(mixed_bird["Common"]))]

In [73]:
unlabeled_mixed_bird["TIME"] = unlabeled_mixed_bird["Length"].apply(lambda x : int(x.split(":")[0]) * 60 + int(x.split(":")[1]))

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [74]:
unlabeled_mixed_bird

,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,Actions,Cat.nr.,copyright,filename,download url,Common,ID,TIME
120,Grey Tinamou (Tinamus tao tao),0:50,GABRIEL LEITE,2020-10-13,06:30,Brazil,"Parauapebas, Pará",500,"adult, life stage uncertain, sex uncertain, song",bird-seen:no\nplayback-used:no\n [sono],ABCDE,XC597722,Creative Commons Attribution-NonCommercial-Sha...,XC597722 - Grey Tinamou - Tinamus tao tao.mp3,/597722/download,Grey Tinamou,597722,50
121,Grey Tinamou (Tinamus tao),0:27,Fabrice Schmitt,2016-02-19,07:30,Colombia,"RNA El Dorado, Sierra de Santa Marta, Magdalena",2000,song,bird-seen:no\nplayback-used:no\n [sono],ABCDE,XC318739,Creative Commons Attribution-NonCommercial-Sha...,XC318739 - Grey Tinamou - Tinamus tao.mp3,/318739/download,Grey Tinamou,318739,27
125,Grey Tinamou (Tinamus tao kleei),0:08,Richard E. Webster,2021-11-24,10:05,Bolivia,"Refugio Volcanes, Samaipata, Florida, Santa Cr...",1100,"adult, sex uncertain, song",bird-seen:no\nplayback-used:no\n [sono],ABCDE,XC708505,Creative Commons Attribution-NonCommercial-Sha...,XC708505 - Grey Tinamou - Tinamus tao kleei.mp3,/708505/download,Grey Tinamou,708505,8
126,Grey Tinamou (Tinamus tao),0:06,Dante Buzzetti,2008-08-04,19:53,Brazil,"Parque Estadual Chandless,Santa Rosa do Purus,...",265,song,"DCB 626-3e, two individuals singing\nbird-seen...",ABCDE,XC564679,Creative Commons Attribution-NonCommercial-Sha...,XC564679 - Grey Tinamou - Tinamus tao.mp3,/564679/download,Grey Tinamou,564679,6
127,Grey Tinamou (Tinamus tao),0:48,Dante Buzzetti,2008-08-04,18:53,Brazil,"Parque Estadual Chandless,Santa Rosa do Purus,...",265,song,"DCB 625-18a, two or three individuals singing....",ABCDE,XC557622,Creative Commons Attribution-NonCommercial-Sha...,XC557622 - Grey Tinamou - Tinamus tao.mp3,/557622/download,Grey Tinamou,557622,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686010,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:52,Eduardo Patrial,2013-10-15,10:00,Brazil,"Cristalino Jungle Lodge, MT",260,song,bird-seen:yes\nplayback-used:yes\n [sono],ABCDE,XC255003,Creative Commons Attribution-NonCommercial-Sha...,XC255003 - Yellow-shouldered Grosbeak - Parker...,/255003/download,Yellow-shouldered Grosbeak,255003,52
686011,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:09,A. Bennett Hennessey,1998-09-25,8:37,Bolivia,"Serranías Beu & Chepete, Pilón Lajas Biospher...",750 - 1450,call,Two birds. At 700 - 1000 m.\n [CD-ROM] [sono],ABCDE,XC3123,Creative Commons Attribution-NonCommercial-NoD...,XC3123 - Yellow-shouldered Grosbeak - Parkerth...,/3123/download,Yellow-shouldered Grosbeak,3123,9
686012,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:37,Roger Ahlman,2011-08-14,16:50,Ecuador,"Miazi, Zamora-Chinchipe",940,"call, song",After playback. Same bird as XC86226.\n [also...,ABCDE,XC93438,Creative Commons Attribution-NonCommercial-NoD...,XC93438 - Yellow-shouldered Grosbeak - Parkert...,/93438/download,Yellow-shouldered Grosbeak,93438,37
686013,Yellow-shouldered Grosbeak (Parkerthraustes hu...,0:31,Huw Lloyd,2008-06-15,16:30,Peru,"Manu Wildlife Centre, Madre de Dios",320,Response to playback,Calls from three birds (possibly parents plus ...,ABCDE,XC36455,Creative Commons Attribution-NonCommercial-NoD...,XC36455 - Yellow-shouldered Grosbeak - Parkert...,/36455/download,Yellow-shouldered Grosbeak,36455,31


In [75]:
counts_of_unlabeled_mixed_bird = unlabeled_mixed_bird.groupby("Common").sum()

In [80]:
possible_species = counts_of_unlabeled_mixed_bird[counts_of_unlabeled_mixed_bird["TIME"] < 500]

In [81]:
possible_species

,TIME
Common,
Agami Heron,199
Amazonian Umbrellabird,373
Amethyst Woodstar,153
Andean Ibis,222
Andean Parakeet,390
...,...
White-tailed Goldenthroat,401
White-throated Jacamar,273
White-tufted Grebe,109


In [84]:
possible_species_files = unlabeled_mixed_bird[unlabeled_mixed_bird["Common"].isin(possible_species.index)]

In [93]:
mixed_bird_counts = mixed_bird.groupby("Common").count().sort_values("IN FILE", ascending=True)
mixed_bird_counts[mixed_bird_counts["IN FILE"] < 25]

,Unnamed: 0,IN FILE,CLIP LENGTH,OFFSET,DURATION,MAX FREQ,MIN FREQ,SAMPLE RATE,MANUAL ID,TIME_SPENT,LAST MOD BY,LAST MOD DATE,LAST MOD TIME,FOLDER,ID,MANUAL ID LENGTH
Common,,,,,,,,,,,,,,,,
Common Gallinule,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Nacunda Nighthawk,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Spangled Cotinga,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Yellow-browed Tyrant,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Sira Curassow,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pectoral Sandpiper,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24
Greater Yellowlegs,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24
Andean Ibis,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24


In [100]:
mixed_bird_counts = mixed_bird.groupby("Common").sum().sort_values("DURATION", ascending=True)
mixed_bird_counts = mixed_bird_counts[["DURATION"]]
mixed_bird_counts

,DURATION
Common,
Sira Curassow,0.2700
Yellow-browed Tyrant,0.4072
Nacunda Nighthawk,0.4713
Spangled Cotinga,0.4886
Mottle-backed Elaenia,0.7481
...,...
Blue-chinned Sapphire,194.9705
White-throated Toucan,218.2865
Ihering's Antwren,230.2584


In [103]:
mixed_bird_counts2 = mixed_bird.groupby("Common").count().sort_values("IN FILE", ascending=True)
mixed_bird_counts2 = mixed_bird_counts2[["IN FILE"]]
mixed_bird_counts = mixed_bird_counts.merge(mixed_bird_counts2, left_index=True, right_index=True)

In [106]:
mixed_bird_counts["FILES_PER_SECOND"] = mixed_bird_counts["IN FILE"] / mixed_bird_counts["DURATION"]
mixed_bird_counts

,DURATION,IN FILE,FILES_PER_SECOND
Common,,,
Sira Curassow,0.2700,1,3.703704
Yellow-browed Tyrant,0.4072,1,2.455796
Nacunda Nighthawk,0.4713,1,2.121791
Spangled Cotinga,0.4886,1,2.046664
Mottle-backed Elaenia,0.7481,2,2.673439
...,...,...,...
Blue-chinned Sapphire,194.9705,48,0.246191
White-throated Toucan,218.2865,164,0.751306
Ihering's Antwren,230.2584,67,0.290977


In [148]:
mixed_bird_counts.head(50)

,DURATION,IN FILE,FILES_PER_SECOND
Common,,,
Sira Curassow,0.2700,1,3.703704
Yellow-browed Tyrant,0.4072,1,2.455796
Nacunda Nighthawk,0.4713,1,2.121791
Spangled Cotinga,0.4886,1,2.046664
Mottle-backed Elaenia,0.7481,2,2.673439
Bare-faced Ground Dove,0.9164,1,1.091227
White-bellied Woodstar,1.1245,4,3.557137
Festive Coquette,1.1268,8,7.099752
Crested Eagle,1.1621,2,1.721022


In [108]:
sum(mixed_bird_counts["FILES_PER_SECOND"])/mixed_bird_counts.shape[0] * 100

116.47292888084444

In [131]:
mixed_bird_counts[(mixed_bird_counts["DURATION"] < 75)].head(30).index

Index(['Sira Curassow', 'Yellow-browed Tyrant', 'Nacunda Nighthawk',
       'Spangled Cotinga', 'Mottle-backed Elaenia', 'Bare-faced Ground Dove',
       'White-bellied Woodstar', 'Festive Coquette', 'Crested Eagle',
       'Azure Gallinule', 'Violet-throated Starfrontlet', 'Screaming Piha',
       'White-tailed Hawk', 'Black-capped Tinamou', 'Yellow-bellied Dacnis',
       'Koepcke's Hermit', 'Varzea Thrush', 'Red-crested Cotinga',
       'White-necked Jacobin', 'Unicolored Blackbird',
       'Hazel-fronted Pygmy Tyrant', 'Great Horned Owl',
       'Plum-throated Cotinga', 'Stripe-backed Bittern',
       'Inambari Woodcreeper', 'White-browed Hawk', 'Alder Flycatcher',
       'White-winged Black Tyrant', 'Maguari Stork', 'Amazonian Umbrellabird'],
      dtype='object', name='Common')

In [142]:
files_to_download= unlabeled_mixed_bird[unlabeled_mixed_bird["Common"].isin(mixed_bird_counts[(mixed_bird_counts["DURATION"] < 75)].head(20).index)]

In [143]:
sum(files_to_download["TIME"])

24568

In [144]:
(60 * 60)/3 * 10


12000.0

In [145]:
files_to_download

,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,Actions,Cat.nr.,copyright,filename,download url,Common,ID,TIME
1738,Black-capped Tinamou (Crypturellus atrocapillus),0:07,Jacob Wijpkema,2021-10-15,17:18,Bolivia,Extrema,260,call,Riverine/gallery forest nearly all large trees...,ABCDE,XC686807,Creative Commons Attribution-NonCommercial-Sha...,XC686807 - Black-capped Tinamou - Crypturellus...,/686807/download,Black-capped Tinamou,686807,7
1739,Black-capped Tinamou (Crypturellus atrocapillus),0:37,Felipe Arantes,2019-05-09,17:30,Brazil,"Parque estadual Chandless, State of Acre",180,song,bird-seen:no\nplayback-used:no\n [sono],ABCDE,XC475704,Creative Commons Attribution-NonCommercial-Sha...,XC475704 - Black-capped Tinamou - Crypturellus...,/475704/download,Black-capped Tinamou,475704,37
1740,Black-capped Tinamou (Crypturellus atrocapillu...,0:25,GABRIEL LEITE,2019-05-07,06:30,Brazil,"Rio Chandless - Parque Estadual Chandless, Man...",180,song,bird-seen:no\nplayback-used:no\n [sono],ABCDE,XC474806,Creative Commons Attribution-NonCommercial-Sha...,XC474806 - Black-capped Tinamou - Crypturellus...,/474806/download,Black-capped Tinamou,474806,25
1741,Black-capped Tinamou (Crypturellus atrocapillu...,2:22,Guy Kirwan,2018-08-01,16:30,Brazil,"Ramal do Noca, Rio Branco, Acre",200,song,bird-seen:no\nplayback-used:yes\n [sono],ABCDE,XC437373,Creative Commons Attribution-NonCommercial-Sha...,XC437373 - Black-capped Tinamou - Crypturellus...,/437373/download,Black-capped Tinamou,437373,142
1742,Black-capped Tinamou (Crypturellus atrocapillus),0:54,Peter Boesman,2009-07-21,08:00,Peru,"Manu Amazonia Lodge, Cuzco",200 - 500,song,ID certainty 100%. (Archiv. tape 542 side A tr...,ABCDE,XC221800,Creative Commons Attribution-NonCommercial-NoD...,XC221800 - Black-capped Tinamou - Crypturellus...,/221800/download,Black-capped Tinamou,221800,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675723,Yellow-bellied Dacnis (Dacnis flaviventer),0:03,Andrew Spencer,2009-09-23,12:00,Brazil,"Cristalino Jungle Lodge, MT",250,call,calls in flight\n [sono],ABCDE,XC39119,Creative Commons Attribution-NonCommercial-NoD...,XC39119 - Yellow-bellied Dacnis - Dacnis flavi...,/39119/download,Yellow-bellied Dacnis,39119,3
675724,Yellow-bellied Dacnis (Dacnis flaviventer),0:51,Andrew Spencer,2009-09-07,07:00,Brazil,"Cristalino Jungle Lodge, MT",250,song,an unfortunate amount of heavy water noise fro...,ABCDE,XC38580,Creative Commons Attribution-NonCommercial-NoD...,XC38580 - Yellow-bellied Dacnis - Dacnis flavi...,/38580/download,Yellow-bellied Dacnis,38580,51
675725,Yellow-bellied Dacnis (Dacnis flaviventer),0:48,Caio Brito,2018-07-04,12:15,Brazil,Tabajara - Igarapé do Marmelo - MD do Machado,100,chamado,bird-seen:yes\nplayback-used:no\n [sono],ABCDE,XC556273,Creative Commons Attribution-NonCommercial-Sha...,XC556273 - Yellow-bellied Dacnis - Dacnis flav...,/556273/download,Yellow-bellied Dacnis,556273,48
675726,Yellow-bellied Dacnis (Dacnis flaviventer),0:48,Caio Brito,2018-07-04,12:15,Brazil,Tabajara - Igarapé do Marmelo - MD do Machado,100,chamado,bird-seen:yes\nplayback-used:no\n [sono],ABCDE,XC556272,Creative Commons Attribution-NonCommercial-Sha...,XC556272 - Yellow-bellied Dacnis - Dacnis flav...,/556272/download,Yellow-bellied Dacnis,556272,48


In [147]:
meta = download_data(files_to_download, -1, "./extra_mixed_bird/")
meta

downloading data
filename        XC686807 - Black-capped Tinamou - Crypturellus...
download url                                     /686807/download
Name: 1738, dtype: object
filename        XC475704 - Black-capped Tinamou - Crypturellus...
download url                                     /475704/download
Name: 1739, dtype: object
filename        XC474806 - Black-capped Tinamou - Crypturellus...
download url                                     /474806/download
Name: 1740, dtype: object
filename        XC437373 - Black-capped Tinamou - Crypturellus...
download url                                     /437373/download
Name: 1741, dtype: object
filename        XC221800 - Black-capped Tinamou - Crypturellus...
download url                                     /221800/download
Name: 1742, dtype: object
filename        XC151848 - Black-capped Tinamou - Crypturellus...
download url                                     /151848/download
Name: 1743, dtype: object
filename        XC122784 - Black-ca

filename        XC53434 - Nacunda Nighthawk - Chordeiles nacun...
download url                                      /53434/download
Name: 27204, dtype: object
filename        XC577516 - Nacunda Nighthawk - Chordeiles nacu...
download url                                     /577516/download
Name: 27205, dtype: object
filename        XC47836 - Koepcke's Hermit - Phaethornis koepc...
download url                                      /47836/download
Name: 35242, dtype: object
filename        XC682197 - White-necked Jacobin - Florisuga me...
download url                                     /682197/download
Name: 36335, dtype: object
filename        XC424604 - White-necked Jacobin - Florisuga me...
download url                                     /424604/download
Name: 36336, dtype: object
filename        XC387213 - White-necked Jacobin - Florisuga me...
download url                                     /387213/download
Name: 36337, dtype: object
filename        XC353894 - White-necked Jacobi

filename        XC221201 - Violet-throated Starfrontlet - Coel...
download url                                     /221201/download
Name: 41008, dtype: object
filename        XC221200 - Violet-throated Starfrontlet - Coel...
download url                                     /221200/download
Name: 41009, dtype: object
filename        XC175872 - Violet-throated Starfrontlet - Coel...
download url                                     /175872/download
Name: 41010, dtype: object
filename        XC2081 - Violet-throated Starfrontlet - Coelig...
download url                                       /2081/download
Name: 41011, dtype: object
filename        XC145018 - Violet-throated Starfrontlet - Coel...
download url                                     /145018/download
Name: 41012, dtype: object
filename        XC36511 - Violet-throated Starfrontlet - Coeli...
download url                                      /36511/download
Name: 41013, dtype: object
filename        XC22969 - Violet-throated Star

filename        XC472242 - Crested Eagle - Morphnus guianensis...
download url                                     /472242/download
Name: 129892, dtype: object
filename        XC614730 - Crested Eagle - Morphnus guianensis...
download url                                     /614730/download
Name: 129894, dtype: object
filename        XC494080 - Crested Eagle - Morphnus guianensis...
download url                                     /494080/download
Name: 129895, dtype: object
filename        XC387284 - Crested Eagle - Morphnus guianensis...
download url                                     /387284/download
Name: 129896, dtype: object
filename        XC387281 - Crested Eagle - Morphnus guianensis...
download url                                     /387281/download
Name: 129897, dtype: object
filename        XC118821 - Crested Eagle - Morphnus guianensis...
download url                                     /118821/download
Name: 129899, dtype: object
filename        XC387282 - Crested Eagle

filename        XC488806 - White-tailed Hawk - Geranoaetus alb...
download url                                     /488806/download
Name: 135987, dtype: object
filename        XC385597 - White-tailed Hawk - Geranoaetus alb...
download url                                     /385597/download
Name: 135988, dtype: object
filename        XC85826 - White-tailed Hawk - Geranoaetus albi...
download url                                      /85826/download
Name: 135989, dtype: object
filename        XC50666 - White-tailed Hawk - Geranoaetus albi...
download url                                      /50666/download
Name: 135990, dtype: object
filename        XC43360 - White-tailed Hawk - Geranoaetus albi...
download url                                      /43360/download
Name: 135991, dtype: object
filename        XC43359 - White-tailed Hawk - Geranoaetus albi...
download url                                      /43359/download
Name: 135992, dtype: object
filename        XC43358 - White-tailed H

...

filename        XC119390 - Screaming Piha - Lipaugus vociferan...
download url                                     /119390/download
Name: 292730, dtype: object
filename        XC106549 - Screaming Piha - Lipaugus vociferan...
download url                                     /106549/download
Name: 292731, dtype: object
filename        XC102838 - Screaming Piha - Lipaugus vociferan...
download url                                     /102838/download
Name: 292732, dtype: object
filename        XC93703 - Screaming Piha - Lipaugus vociferans...
download url                                      /93703/download
Name: 292733, dtype: object
filename        XC88353 - Screaming Piha - Lipaugus vociferans...
download url                                      /88353/download
Name: 292734, dtype: object
filename        XC84112 - Screaming Piha - Lipaugus vociferans...
download url                                      /84112/download
Name: 292735, dtype: object
filename        XC75016 - Screaming Piha

filename        XC102397 - Screaming Piha - Lipaugus vociferan...
download url                                     /102397/download
Name: 292782, dtype: object
filename        XC72670 - Screaming Piha - Lipaugus vociferans...
download url                                      /72670/download
Name: 292783, dtype: object
filename        XC68571 - Screaming Piha - Lipaugus vociferans...
download url                                      /68571/download
Name: 292784, dtype: object
filename        XC64950 - Screaming Piha - Lipaugus vociferans...
download url                                      /64950/download
Name: 292785, dtype: object
filename        XC61265 - Screaming Piha - Lipaugus vociferans...
download url                                      /61265/download
Name: 292786, dtype: object
filename        XC59794 - Screaming Piha - Lipaugus vociferans...
download url                                      /59794/download
Name: 292787, dtype: object
filename        XC59793 - Screaming Piha

filename        XC454262 - Unicolored Blackbird - Agelasticus ...
download url                                     /454262/download
Name: 649022, dtype: object
filename        XC452687 - Unicolored Blackbird - Agelasticus ...
download url                                     /452687/download
Name: 649023, dtype: object
filename        XC435780 - Unicolored Blackbird - Agelasticus ...
download url                                     /435780/download
Name: 649024, dtype: object
filename        XC431264 - Unicolored Blackbird - Agelasticus ...
download url                                     /431264/download
Name: 649025, dtype: object
filename        XC427044 - Unicolored Blackbird - Agelasticus ...
download url                                     /427044/download
Name: 649026, dtype: object
filename        XC347857 - Unicolored Blackbird - Agelasticus ...
download url                                     /347857/download
Name: 649027, dtype: object
filename        XC84389 - Unicolored Bla

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Common name / Scientific,Length,Recordist,Date,Time,Country,Location,Elev. (m),Type,Remarks,Actions,Cat.nr.,copyright,filename,download url,Common,ID,TIME,file_location
1738,Black-capped Tinamou (Crypturellus atrocapillus),0:07,Jacob Wijpkema,2021-10-15,17:18,Bolivia,Extrema,260,call,Riverine/gallery forest nearly all large trees...,ABCDE,XC686807,Creative Commons Attribution-NonCommercial-Sha...,XC686807 - Black-capped Tinamou - Crypturellus...,/686807/download,Black-capped Tinamou,686807,7,./extra_mixed_bird/XC686807 - Black-capped Tin...
1739,Black-capped Tinamou (Crypturellus atrocapillus),0:37,Felipe Arantes,2019-05-09,17:30,Brazil,"Parque estadual Chandless, State of Acre",180,song,bird-seen:no\nplayback-used:no\n [sono],ABCDE,XC475704,Creative Commons Attribution-NonCommercial-Sha...,XC475704 - Black-capped Tinamou - Crypturellus...,/475704/download,Black-capped Tinamou,475704,37,./extra_mixed_bird/XC475704 - Black-capped Tin...
1740,Black-capped Tinamou (Crypturellus atrocapillu...,0:25,GABRIEL LEITE,2019-05-07,06:30,Brazil,"Rio Chandless - Parque Estadual Chandless, Man...",180,song,bird-seen:no\nplayback-used:no\n [sono],ABCDE,XC474806,Creative Commons Attribution-NonCommercial-Sha...,XC474806 - Black-capped Tinamou - Crypturellus...,/474806/download,Black-capped Tinamou,474806,25,./extra_mixed_bird/XC474806 - Black-capped Tin...
1741,Black-capped Tinamou (Crypturellus atrocapillu...,2:22,Guy Kirwan,2018-08-01,16:30,Brazil,"Ramal do Noca, Rio Branco, Acre",200,song,bird-seen:no\nplayback-used:yes\n [sono],ABCDE,XC437373,Creative Commons Attribution-NonCommercial-Sha...,XC437373 - Black-capped Tinamou - Crypturellus...,/437373/download,Black-capped Tinamou,437373,142,./extra_mixed_bird/XC437373 - Black-capped Tin...
1742,Black-capped Tinamou (Crypturellus atrocapillus),0:54,Peter Boesman,2009-07-21,08:00,Peru,"Manu Amazonia Lodge, Cuzco",200 - 500,song,ID certainty 100%. (Archiv. tape 542 side A tr...,ABCDE,XC221800,Creative Commons Attribution-NonCommercial-NoD...,XC221800 - Black-capped Tinamou - Crypturellus...,/221800/download,Black-capped Tinamou,221800,54,./extra_mixed_bird/XC221800 - Black-capped Tin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675723,Yellow-bellied Dacnis (Dacnis flaviventer),0:03,Andrew Spencer,2009-09-23,12:00,Brazil,"Cristalino Jungle Lodge, MT",250,call,calls in flight\n [sono],ABCDE,XC39119,Creative Commons Attribution-NonCommercial-NoD...,XC39119 - Yellow-bellied Dacnis - Dacnis flavi...,/39119/download,Yellow-bellied Dacnis,39119,3,./extra_mixed_bird/XC39119 - Yellow-bellied Da...
675724,Yellow-bellied Dacnis (Dacnis flaviventer),0:51,Andrew Spencer,2009-09-07,07:00,Brazil,"Cristalino Jungle Lodge, MT",250,song,an unfortunate amount of heavy water noise fro...,ABCDE,XC38580,Creative Commons Attribution-NonCommercial-NoD...,XC38580 - Yellow-bellied Dacnis - Dacnis flavi...,/38580/download,Yellow-bellied Dacnis,38580,51,./extra_mixed_bird/XC38580 - Yellow-bellied Da...
675725,Yellow-bellied Dacnis (Dacnis flaviventer),0:48,Caio Brito,2018-07-04,12:15,Brazil,Tabajara - Igarapé do Marmelo - MD do Machado,100,chamado,bird-seen:yes\nplayback-used:no\n [sono],ABCDE,XC556273,Creative Commons Attribution-NonCommercial-Sha...,XC556273 - Yellow-bellied Dacnis - Dacnis flav...,/556273/download,Yellow-bellied Dacnis,556273,48,./extra_mixed_bird/XC556273 - Yellow-bellied D...
675726,Yellow-bellied Dacnis (Dacnis flaviventer),0:48,Caio Brito,2018-07-04,12:15,Brazil,Tabajara - Igarapé do Marmelo - MD do Machado,100,chamado,bird-seen:yes\nplayback-used:no\n [sono],ABCDE,XC556272,Creative Commons Attribution-NonCommercial-Sha...,XC556272 - Yellow-bellied Dacnis - Dacnis flav...,/556272/download,Yellow-bellied Dacnis,556272,48,./extra_mixed_bird/XC556272 - Yellow-bellied D...


In [ ]:
mixed_bird["Common"] = mixed_bird["MANUAL ID"].apply(lambda x: " ".join(x.split(" ")[2:]))
mixed_bird